In [1]:
from cnn_source import Simple3DCNN
import torch
from torchvision import transforms
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
#device = torch.device('cpu')
squat_model_file_1 = 'model/squat_train.pt'
#model_squat_1 = Simple3DCNN().to(device)
#model_squat_1.load_state_dict(torch.load(squat_model_file_1,map_location=device))
model_file = 'model/classifying_cnn.pt'
model = Simple3DCNN().to(device)
model.load_state_dict(torch.load(model_file,map_location=device))

C:\Users\JPJ\AppData\Local\Temp\ipykernel_15476\3073132621.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_file,map_location=devi

<All keys matched successfully>

In [2]:
import cv2
import numpy as np
import torch

# 실시간 데이터 준비 함수
def prepare_realtime_data(frame, frames_per_video=15, height=640, width=480, device='cpu'):
    frames = []

    while len(frames) < frames_per_video:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        green_frame = frame[:, :, 1]  # 초록색 채널 추출 (H, W)
        green_frame = cv2.resize(green_frame, (width, height))  # 크기 조정
        frames.append(green_frame)

    # 부족한 프레임을 0으로 패딩
    while len(frames) < frames_per_video:
        frames.append(np.zeros((height, width), dtype=np.float32))

    # numpy 배열 -> torch 텐서 변환 및 차원 조정
    frames = torch.tensor(np.array(frames), dtype=torch.float32, device=device)  # (T, H, W)
    frames = frames.unsqueeze(1)  # (T, H, W) -> (T, C, H, W)
    frames = frames.permute(1, 0, 2, 3).unsqueeze(0)  # (T, C, H, W) -> (1, T, C, H, W)

    return frames


In [3]:
# 데이터 전처리 정의 (프레임 단위로 처리)
transform = transforms.Compose([
    transforms.ToPILImage(),                  # Numpy 배열을 PIL 이미지로 변환          
    transforms.ToTensor(),
    transforms.Resize((640,480))          
])

In [ ]:
import cv2
import torch
import numpy as np
import time
import mediapipe as mp

# Mediapipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5)

# 모델 로드
model.eval()  # 평가 모드 설정

# 카메라 캡처 객체 생성
cap = cv2.VideoCapture(1)  # 첫 번째 카메라 사용

if not cap.isOpened():
    print("Error: Cannot access the camera.")
    exit()

# 카메라 해상도 설정
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 주요 파라미터
frames_per_video = 15
height, width = 480, 640

# 카운트다운 시작
countdown = 5
while countdown > 0:
    ret, frame = cap.read()
    #frame = cv2.resize(frame,(640,480))
    if not ret:
        print("Error: Failed to capture image.")
        break

    # BGR 이미지를 RGB로 변환 후 처리
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 검은 화면 생성
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)),
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

    # 화면 크기
    height, width, _ = frame.shape

    # 텍스트 설정
    text = str(countdown)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 3
    font_thickness = 5
    color = (0, 255, 0)

    # 텍스트 중앙에 배치
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    text_x = (width - text_size[0]) // 2
    text_y = (height + text_size[1]) // 2

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

    # 텍스트 추가
    cv2.putText(frame, text, (text_x, text_y), font, font_scale, color, font_thickness)

    # 화면 표시
    # cv2.imshow("Pose Estimation", frame)
    countdown -= 1
    time.sleep(1)  # 잠시 대기

# 실시간 추론 루프
print("Starting pose estimation...")
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    # BGR 이미지를 RGB로 변환 후 처리
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 검은 화면 생성
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
    if not results.pose_landmarks:
        print("no person to make pose graph")
        break
    # 실시간 데이터 준비
    frames = prepare_realtime_data(black_frame, frames_per_video, height, width, device)

    # 모델 추론
    with torch.no_grad():
        outputs = model(frames)
        predicted_class = torch.argmax(outputs).item()

    # 결과 표시
    if predicted_class == 0:
        pose_pr = "Push-up"
    elif predicted_class == 1:
        pose_pr = "Squat"
    elif predicted_class == 2:
        pose_pr = "Lunge"
    else:
        pose_pr = "Unknown"

    # 화면에 포즈 결과 표시
    cv2.putText(frame, f"Pose: {pose_pr}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # 화면 표시
    cv2.imshow("Pose Estimation", frame)

    # 'q'를 눌러 종료
    if cv2.waitKey(1) == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [4]:
import sys
import time
import cv2
import torch
import numpy as np
import mediapipe as mp
from PyQt5 import QtCore, QtGui, QtWidgets
from cnn_source import Simple3DCNN
import torch
from torchvision import transforms
import torch.nn as nn

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')

model_file = 'model/classifying_cnn.pt'
model = Simple3DCNN().to(device)
model.load_state_dict(torch.load(model_file, map_location=device))

# Mediapipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5)

model.eval()  # 평가 모드 설정

# 실시간 데이터 준비 함수
def prepare_realtime_data(frame, frames_per_video=15, height=640, width=480, device='cpu'):
    frames = []

    while len(frames) < frames_per_video:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
        green_frame = frame[:, :, 1]  # 초록색 채널 추출 (H, W)
        green_frame = cv2.resize(green_frame, (width, height))  # 크기 조정
        frames.append(green_frame)

    # 부족한 프레임을 0으로 패딩
    while len(frames) < frames_per_video:
        frames.append(np.zeros((height, width), dtype=np.float32))

    # numpy 배열 -> torch 텐서 변환 및 차원 조정
    frames = torch.tensor(np.array(frames), dtype=torch.float32, device=device)  # (T, H, W)
    frames = frames.unsqueeze(1)  # (T, H, W) -> (T, C, H, W)
    frames = frames.permute(1, 0, 2, 3).unsqueeze(0)  # (T, C, H, W) -> (1, T, C, H, W)

    return frames

# PyQt5 윈도우 클래스
class PoseEstimationApp(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Pose Estimation")
        self.setGeometry(100, 100, 800, 600)

        # UI 설정
        self.start_button = QtWidgets.QPushButton("Start", self)
        self.quit_button = QtWidgets.QPushButton("Quit", self)
        self.label = QtWidgets.QLabel("Pose: Unknown", self)
        self.countdown_label = QtWidgets.QLabel("Countdown: 5", self)

        # 레이아웃 설정
        self.layout = QtWidgets.QVBoxLayout()
        self.layout.addWidget(self.start_button)
        self.layout.addWidget(self.quit_button)
        self.layout.addWidget(self.label)
        self.layout.addWidget(self.countdown_label)
        self.setLayout(self.layout)

        # 버튼 클릭 이벤트 연결
        self.start_button.clicked.connect(self.start_pose_estimation)
        self.quit_button.clicked.connect(self.quit_program)

        # 웹캠 캡처 객체 초기화
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Cannot access the camera.")
            sys.exit()

        # 웹캠 해상도 설정
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        self.timer = QtCore.QTimer(self)
        self.timer.timeout.connect(self.update_countdown)

        self.countdown = 5
        self.is_recording = False
        self.frame = None

    def start_pose_estimation(self):
        self.start_button.setEnabled(False)
        self.countdown = 5
        self.is_recording = False
        self.timer.start(1000)  # 1초 간격으로 타이머 시작

    def update_countdown(self):
        if self.countdown > 0:
            self.countdown_label.setText(f"Countdown: {self.countdown}")
            self.countdown -= 1
        else:
            self.timer.stop()
            self.is_recording = True
            self.label.setText("Starting pose estimation...")
            self.countdown_label.setText("Recording...")
            self.timer.timeout.disconnect()
            self.timer.timeout.connect(self.update_frame)
            self.timer.start(30)  # 프레임 업데이트를 위한 30ms 간격 타이머

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            self.timer.stop()

        self.frame = frame
        if self.is_recording:
            # 포즈 추정 수행
            results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            black_frame = np.zeros((480, 640, 3), dtype=np.uint8)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

            # 실시간 데이터 준비
            frames = prepare_realtime_data(black_frame)

            # 모델 추론
            with torch.no_grad():
                outputs = model(frames)
                predicted_class = torch.argmax(outputs).item()

            # 결과 표시
            pose_pr = {0: "Push-up", 1: "Squat", 2: "Lunge"}.get(predicted_class, "Unknown")
            self.label.setText(f"Pose: {pose_pr}")
            cv2.putText(frame, f"Pose: {pose_pr}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # 웹캠 영상 보여주기
        self.display_image(frame)

    def display_image(self, img):
        # OpenCV에서 QImage로 변환
        rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        q_img = QtGui.QImage(rgb_image.data, w, h, bytes_per_line, QtGui.QImage.Format_RGB888)

        # QLabel에 이미지를 표시
        self.label.setPixmap(QtGui.QPixmap.fromImage(q_img))

    def quit_program(self):
        self.cap.release()
        self.close()


# 실행
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    window = PoseEstimationApp()
    window.show()
    sys.exit(app.exec_())


C:\Users\JPJ\AppData\Local\Temp\ipykernel_15476\525869798.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_file, map_location=devi

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

...

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

Error: Failed to capture image.


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


SystemExit: 0

c:\Users\JPJ\PycharmProjects\sqc\AI_project\oldjins\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
